# Final Scoring

## Introduction
This takes the keyword_scored.csv which has the output of all the metrics evaluated so far. This notebook finds the final scores and values from the metrics computed so far

The WEIGHT_METRICS determines whether weighted values or unweighted values should be used to determine the final scores and voices

In [ ]:
WEIGHT_METRICS = True

In [11]:
import pandas as pd
df = pd.read_csv("./keyterm_scored.csv")
del df['idx']
df.clauses_text_final = df.clauses_text_final.apply(eval)
df.voice = df.voice.apply(eval)
df.abstraction_score_normalized = df.abstraction_score_normalized.apply(eval)
df.reading_ease_normalized = df.reading_ease_normalized.apply(eval)
df.grading_level_normalized = df.grading_level_normalized.apply(eval) 
df.sgrank_normalized = df.sgrank_normalized.apply(eval)
df.sample(frac = 1).head(20)

,prompt,response,clauses_text_final,voice,abstraction_score_normalized,reading_ease_normalized,grading_level_normalized,sgrank_normalized
141,My friends and I,have a very close relationship. Although I may...,"[have a very close relationship, Although I ma...","[P_bevb_x, P_bevb_x, A_pron_x, A_pron_x, A_def...","[1.0, 0.88, 1.0, 1.0, 0.56, 1.0, 0.56, 0.88, 1...","[0.16, 0.15, 0.08, 0.14, 0.09, 0.32, 0.11, 0.1...","[0.18, 0.18, 0.13, 0.17, 0.1, 0.34, 0.13, 0.19...","[0.06, 0.04, 0.04, 0.06, 0.02, 0.02, 0.02, 0.0..."
531,Change is,"happening all the time, continues and is inevi...","[happening, all the time continues and, is ine...","[A_def, A_def, P_bevb_x, A_pron_x, P_bevb_x, P...","[0.56, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.11, 0.05, 0.24, 0.13, 0.18, 0.22]","[0.5, 0.13, 0.09, 0.32, 0.17, 0.22, 0.29]","[0.0, 0.19, 0.08, 0.08, 0.16, 0.19, 0.1]"
85,We could make the world a better place if,"if we are present, open portal to flow of tran...","[if we are present open portal, to flow of tra...","[P_bevb_x, A_def, A_def, A_pron_x]","[0.56, 1.0, 1.0, 0.56]","[0.06, 0.22, 0.45, 0.26]","[0.08, 0.25, 0.46, 0.28]","[0.1, 0.16, 0.22, 0.27]"
485,These days progress means,letting go while driving forward; finding cont...,"[letting go, letting go, while driving forward...","[A_def, A_def, A_def, A_def, A_def, A_def, A_d...","[0.56, 0.56, 1.0, 1.0, 0.56, 1.0]","[0.13, 0.13, 0.23, 0.36, 0.24, 0.0]","[0.13, 0.13, 0.26, 0.38, 0.28, 0.01]","[0.0, 0.0, 0.09, 0.15, 0.28, 0.0]"
278,"These days, school",really hard.,[hard],[Undefined],[0.56],[0.0],[0.0],[0.0]
326,If I had more money,I would buy a swim with seals.,[I would buy a swim with seals],[P_bevb_x],[0.88],[0.02],[0.05],[0.4]
22,My conscience bothers me if,If my words or actions have hurt someone,[If my words or actions have hurt someone],[A_pron_x],[1.0],[0.08],[0.12],[0.22]
291,At times I worry about,when my toys get lost,[when my toys get lost],[A_pron_x],[0.56],[0.01],[0.03],[0.65]
366,If I had more money,I would spend it on cool stuff.,[I would spend it on cool stuff],[P_bevb_x],[1.0],[0.02],[0.05],[0.65]
343,When they didn't let me join in,I felt sad,[I felt sad],[P_get_x],[0.88],[0.01],[0.02],[0.0]


Each metric value is an array of scores of each clause. For each sentence, the voice array and the arrays of each metric will have the same length, all of which equal to the total count of clauses. In each metric, the maximum value per clause is determined.. We will be using the 0-based index of the maximum value to identify the voice later.

In [12]:
df['idx_abstraction_score'] = df['abstraction_score_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_grading_level'] = df['grading_level_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_reading_ease'] = df['reading_ease_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_sgrank'] = df['sgrank_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)

df[['prompt', 'response', 'clauses_text_final', 'voice', 'idx_abstraction_score', 'idx_grading_level', 'idx_reading_ease', 'idx_sgrank']].sample(frac = 1).head()

,prompt,response,clauses_text_final,voice,idx_abstraction_score,idx_grading_level,idx_reading_ease,idx_sgrank
515,I just can\'t stand people who,there are no people really\nno worth not stand...,"[who there are no people really no worth, not ...","[P_bevb_x, A_def, P_bevb_x]",2,0,0,0
120,Change is,...happening each day on the one side: kids gr...,"[happening each day on the one side kids, grow...","[A_def, A_def, A_def, P_get_x, A_def, P_bevb_x...",0,4,4,12
167,A good boss,identifies with every employee as an expressio...,[identifies with every employee as an expressi...,"[A_def, P_bevb_x, A_def, A_def, P_bevb_x, A_de...",1,17,17,3
366,If I had more money,I would spend it on cool stuff.,[I would spend it on cool stuff],[P_bevb_x],0,0,0,0
376,If I can\'t get what I want,I'm okay,[want],[A_def],0,0,0,0


The voice based on each metric is determined by the voice that corresponds to the maximum-value per metric. So is the final score per metric. These are later used to determine the final score and final voice.

In [13]:
df['voice_abstraction'] = df.apply(lambda row : row['voice'][row['idx_abstraction_score']] if row['idx_abstraction_score'] >=0 else "Indeterminate", axis = 1)
df['voice_grading_level'] = df.apply(lambda row : row['voice'][row['idx_grading_level']] if row['idx_grading_level'] >=0 else "Indeterminate", axis = 1)
df['voice_reading_ease'] = df.apply(lambda row : row['voice'][row['idx_reading_ease']] if row['idx_reading_ease'] >=0 else "Indeterminate", axis = 1)
df['voice_sgrank'] = df.apply(lambda row : row['voice'][row['idx_sgrank']] if row['idx_sgrank'] >=0 else "Indeterminate", axis = 1)

df['final_abstraction'] = df.apply(lambda row : row['abstraction_score_normalized'][row['idx_abstraction_score']] if row['idx_abstraction_score'] >=0 else -1, axis = 1)
df['final_grading_level'] = df.apply(lambda row : row['grading_level_normalized'][row['idx_grading_level']] if row['idx_grading_level'] >=0 else -1, axis = 1)
df['final_reading_ease'] = df.apply(lambda row : row['reading_ease_normalized'][row['idx_reading_ease']] if row['idx_reading_ease'] >=0 else -1, axis = 1)
df['final_sgrank'] = df.apply(lambda row : row['sgrank_normalized'][row['idx_sgrank']] if row['idx_sgrank'] >=0 else -1, axis = 1)

df[['prompt', 'response', 'clauses_text_final', 'voice', 'final_abstraction', 'voice_abstraction', 'final_grading_level', 'voice_grading_level', 'final_reading_ease', 'voice_reading_ease', 'final_sgrank', 'voice_sgrank']].head()
#df.iloc[342]

,prompt,response,clauses_text_final,voice,final_abstraction,voice_abstraction,final_grading_level,voice_grading_level,final_reading_ease,voice_reading_ease,final_sgrank,voice_sgrank
0,The past,"Winds through us, both from our lives and cult...",[Winds through us both from our lives and cult...,"[A_pron_x, A_pron_x, P_bevb_x, P_bevb_x, P_bev...",1.00,A_pron_x,0.42,A_pron_x,0.26,A_pron_x,0.12,P_bevb_x
1,At times I worry about,Insufficient care and attention for his egocen...,[Insufficient care and attention for his egoce...,[A_pron_x],1.00,A_pron_x,0.49,A_pron_x,0.31,A_pron_x,0.08,A_pron_x
2,What gets me into trouble is,not considering others possibilities.,[not considering others possibilities],[A_def],1.00,A_def,0.52,A_def,0.51,A_def,0.00,A_def
3,When people are helpless,They often don&#039;t know it so they flak aro...,"[They often don, t know it, so they flak aroun...","[P_bevb_x, A_def, A_def, A_def]",0.56,P_bevb_x,0.12,A_def,0.10,A_def,0.00,P_bevb_x
4,Privacy,is a sense of hiding from others that which yo...,"[is a sense of, hiding from others that, which...","[P_bevb_x, A_def, P_bevb_x, P_bevb_x, A_def, A...",1.00,P_bevb_x,0.21,A_def,0.20,A_def,0.65,P_bevb_x


#### Difference between voices "Indeterminate" and "Undefined":
When a clause doesn't match any existing rules, it is labelled "Undefined". When there are no valid clauses in the response is labelled "Indeterminate"

#### Weighing vs Unweighing

* If weighing is chosen, the weights for each metric is computed dynamically as the mean of each metric. The weighted values  are then computed as factors of the mean value. The maximum of the final factors determined. The voice based on this metric is the final voice.
* If weighing is not chosen, the same process is used as above. The weights are 1 for all metrics. 
* The final score is determined irrespective of the weights. It is the average of the unweighted values of each metric.

In [14]:
df['sentinel'] = "Indeterminate"

df['abstraction_weight'] = 1/df['final_abstraction'].mean() if WEIGHT_METRICS else 1
df['grading_level_weight'] = 1/df['final_grading_level'].mean() if WEIGHT_METRICS else 1
df['reading_ease_weight'] = 1/df['final_reading_ease'].mean() if WEIGHT_METRICS else 1
df['sgrank_weight'] = 1/df['final_sgrank'].mean() if WEIGHT_METRICS else 1

cdict = {-1 : 'sentinel', 0 : 'voice_abstraction', 1 : 'voice_grading_level', 2 : 'voice_reading_ease', 3 : 'voice_sgrank'}
def get_final_voice(row):
    arr = [row['final_abstraction'] * row['abstraction_weight'], row['final_grading_level'] * row['grading_level_weight'], \
             row['final_reading_ease'] * row['reading_ease_weight'], row['final_sgrank'] * row['sgrank_weight']]
    idx = arr.index(max(arr))
    return row[cdict[idx]]
    
df['final_score'] = df.apply(lambda row: (row['final_abstraction'] + row['final_grading_level'] + row['final_reading_ease'] + row['final_sgrank'])/4 if row['final_abstraction'] >=0 else -1 , axis = 1)
df['final_voice'] = df.apply(get_final_voice, axis = 1)
df.final_score = df.final_score.apply(lambda x : round(x, 2))
df[['response', 'clauses_text_final', 'final_score', 'final_voice']].sample(frac = 1).head(20)

,response,clauses_text_final,final_score,final_voice
249,would be more widely adopted and used if they ...,"[would be, more widely adopted and, used, if t...",0.44,P_bevb_x
66,my palms sweat and my stomach hurts,"[my palms sweat and, my stomach hurts]",0.39,A_pron_x
489,"is a swirl of subtle exchanges, inevitably rea...","[is a swirl of subtle exchanges, inevitably re...",0.39,P_bevb_x
473,are offensive,[who are offensive],0.35,P_bevb_x
420,I would be rich,[I would be rich],0.42,P_bevb_x
45,present and future continuum of time unfolds i...,[present and future continuum of time unfolds ...,0.58,A_def
361,I don't know,"[I don t, know]",0.23,P_bevb_x
212,I fidget. I speak louder. I speak quicker. My ...,"[I fidget, I speak louder, I speak quicker My ...",0.31,A_pron_x
505,"When I am nervous, I drink a lot of water. If ...","[When I am nervous, I drink a lot of water, If...",0.40,P_bevb_x
95,comes in many different forms.,[comes in many different forms],0.47,A_def


The final output provides two files:
1. The debug file. This contains all the original input columns, the internal metrics, the normalized values, the voices based on each metric, the scores based on each metric, the final score and the final voices.
2. The output file contains the prompt, response, final voice and final score without any internal values. 

In [16]:
suffix = "_weighted"  if WEIGHT_METRICS else ""
df[['response', 'clauses_text_final', 'final_score', 'final_voice']].to_csv("./output{}.csv".format(suffix), index = False)
df[['prompt', 'response', 'clauses_text_final', 'voice', 'abstraction_score_normalized', 'reading_ease_normalized', \
       'grading_level_normalized', 'sgrank_normalized',   \
    'abstraction_weight', 'grading_level_weight', 'reading_ease_weight', 'sgrank_weight', \
    'final_abstraction', 'final_grading_level', 'final_reading_ease', 'final_sgrank', 'final_score', \
       'voice_abstraction', 'voice_grading_level', 'voice_reading_ease', 'voice_sgrank', 'final_voice']].to_csv("./debug{}.csv".format(suffix), index = False)

print("Written")

Written
